In [1]:
from torchvision.datasets import CIFAR10
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
import torch
import torch.nn as nn

/home/itrushkin/.virtualenvs/git-re-basin/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SPLIT=True
MODEL_ID=1
BATCH_SIZE=2**9
NUM_EPOCHS=100

In [3]:
torch.manual_seed(MODEL_ID)
np.random.seed(MODEL_ID)
torch.use_deterministic_algorithms(False)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
train_transform = transforms.Compose([
    transforms.RandomAffine(0, scale=[0.8, 1.2]),
    transforms.RandomCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(30),
    transforms.ToTensor()
])

valid_transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = CIFAR10('.', train=True, transform=train_transform, download=True)
valid_dataset = CIFAR10('.', train=False, transform=valid_transform)
def get_non_iid_indices(labels):
    condition = labels < 5 if MODEL_ID == 1 else labels >= 5
    return condition.nonzero()[0]
def get_iid_indices(labels):
    indices = np.random.permutation(len(labels))
    indices = np.array_split(indices, 2)[MODEL_ID]
    return indices

if SPLIT:
    labels = np.array(train_dataset.targets)
    indices = get_iid_indices(labels)
    train_dataset = Subset(train_dataset, indices)


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)

Files already downloaded and verified


In [ ]:
class Conv2dLayerNorm(nn.LayerNorm):        
    def forward(self, x):
        transposed = x.transpose(1,3)
        result = super().forward(transposed)
        
        return result.transpose(3,1)

In [ ]:
def gn(channels):
    return nn.GroupNorm(32, channels)

In [1]:
class AffineInstanceNorm2d(nn.InstanceNorm2d):
    def __init__(self, *args, **kwargs):
        super().__init__(**kwargs, affine=True)

NameError: name 'nn' is not defined

In [6]:
class Net(nn.Module):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 64)
    self.conv2 = nn.Conv2d(64, 64)
    self.conv3 = nn.Conv2d(64, 128)
    self.conv4 = nn.Conv2d(128, 128)
    self.conv5 = nn.Conv2d(128, 256)
    self.conv6 = nn.Conv2d(256, 256)
    self.conv7 = nn.Conv2d(256, 512)
    self.conv8 = nn.Conv2d(512, 512)
    
    
    
model = resnet18(num_classes=10,norm_layer=gn).to(device)    

In [7]:
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR

optimizer = SGD(model.parameters(), momentum=0.9, lr=5e-3)
scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-6)

In [8]:
from torch import nn
loss_fn = nn.CrossEntropyLoss()

In [ ]:
import torch
import torch.nn.functional as F
def train():
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = loss_fn(outputs, target)
        loss.backward()
        optimizer.step()
    scheduler.step()
    return loss.item()

def validate():
    model.eval()
    total_samples = 0
    val_score = 0
    for data, target in valid_loader:
        optimizer.zero_grad()
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = loss_fn(outputs, target)
        pred = outputs.argmax(dim=1)
        val_score += pred.eq(target).sum().cpu().numpy()
        total_samples += len(target)
    return loss.item(), val_score / total_samples

for epoch in range(NUM_EPOCHS):
    train_loss = train()
    val_loss, acc = validate()
    print(f'Epoch {epoch+1}:\tTrain loss: {train_loss:.4f}\tTest loss: {val_loss:.4f}\tTest accuracy: {acc:.4f}')

Epoch 1:	Train loss: 1.942138	Test loss: 1.787994	Test accuracy: 0.324700
Epoch 2:	Train loss: 1.629336	Test loss: 1.605208	Test accuracy: 0.406700
Epoch 3:	Train loss: 1.602989	Test loss: 1.533300	Test accuracy: 0.438900
Epoch 4:	Train loss: 1.507441	Test loss: 1.464249	Test accuracy: 0.462100
Epoch 5:	Train loss: 1.538440	Test loss: 1.445706	Test accuracy: 0.470500
Epoch 6:	Train loss: 1.467821	Test loss: 1.364759	Test accuracy: 0.504800
Epoch 7:	Train loss: 1.429962	Test loss: 1.328985	Test accuracy: 0.505300
Epoch 8:	Train loss: 1.469605	Test loss: 1.286566	Test accuracy: 0.529000
Epoch 9:	Train loss: 1.373327	Test loss: 1.275438	Test accuracy: 0.547200
Epoch 10:	Train loss: 1.325434	Test loss: 1.297026	Test accuracy: 0.544100
Epoch 11:	Train loss: 1.337210	Test loss: 1.263030	Test accuracy: 0.543000
Epoch 12:	Train loss: 1.266734	Test loss: 1.284214	Test accuracy: 0.534300
Epoch 13:	Train loss: 1.350701	Test loss: 1.264096	Test accuracy: 0.550200
Epoch 14:	Train loss: 1.286793	Tes

In [ ]:
import pickle
with open(f'model_{MODEL_ID}.pkl','wb') as f:
    pickle.dump({n: p.numpy(force=True) for n,p in model.state_dict().items()}, f) 

In [ ]:
for n,p in model.state_dict().items():
    print(f'{n}: {p.size()}')

In [ ]:
torch.tensor([64.3]).numpy(force=True)